# 爬取台灣股票資料

已完成有股票時做新增，沒有時則建立新的csv

In [ ]:
import requests
import pandas as pd
import time
import os
from io import StringIO
import warnings
warnings.filterwarnings("ignore")

def truncate(sio):
    sio.truncate(0) # 清空第0個位置之後的資料
    sio.seek(0)  #保險一點多寫用seek從第0個位置開始添加
    return sio

save_file_path = 'my_stock'
stock_list = ['2303','2603','2609']

columns = ["日期","成交股數","成交金額","開盤價","最高價","最低價","收盤價","漲跌價差","成交筆數"]
stock_df =pd.DataFrame(columns = columns)

for stock in stock_list: # 走過每支股票
    start_month = '2021-12'
    end_month = '2022-01'
    
    #確認是否已經有檔案
    if os.path.exists(save_file_path + f'/{stock}.csv'):
        have_csv = True
        print(f'{stock} already exists.    path: {save_file_path}/{stock}.csv')
    else:
        have_csv = False
        print(f'{stock}.csv : no file.    新增檔案-path: {save_file_path}/{stock}.csv')

    if have_csv == True:
        #取最後一筆資料的日期
        stock_df = pd.read_csv(save_file_path + f'/{stock}.csv')  #, encoding = 'utf-8') #用stock_df去接csv的資料
        stock_df = stock_df.drop(['Unnamed: 0'], axis = 1) # 將最前面的無用index刪除
        get_month = stock_df.values[-1][0]
        start_month= (str(int(get_month[0:3])+1911)+'-'+(get_month)[4:6])  # 取到最後一筆資料的月份
        print(f'起始月份 : {start_month}')  #  確認開啟的股票和起始時間
        print(f'結束月份 : {end_month}')    #  確認結束股票月份
    else:  # 這裡是原本沒有資料跑的
        print(f'起始月份 : {start_month}')  #  確認開啟的股票和起始時間
        print(f'結束月份 : {end_month}')    #  確認結束股票月份


    month_range = [str(data).replace('-','').split(' ')[0] for data in pd.date_range(start = start_month, end = end_month, freq = 'MS')]
    for m in month_range:  # 走過每個月份
        print(f'正在執行 : {m}'   , end="")
        url = 'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date='+ m +'&stockNo='+stock
        print(' - processing:',url)
        headers = {'User-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
        res = requests.get(url, headers = headers)
        res.text

        io_csv_info = StringIO(res.text)
        io_csv_info.readline()  #讀取第一行(股票代號資訊我們不需要)
        io_csv_info.readline()  # 不需要 '"日期","成交股數","成交金額","開盤價","最高價","最低價","收盤價","漲跌價差","成交筆數",\r\n'

        # 接下來開始是需要的資訊，所以要用while迴圈處理
        while True:  # 走過每一天
            line = io_csv_info.readline()
            info_list = line.split('","')
            info_list[0] = info_list[0].strip('"')
            info_list[-1] = info_list[-1].strip('",\r\n')
            if '"說明:"\r\n' in line:  # 當讀到說明欄位，則跳出
                break
                
            if have_csv == True : # 有資料的話要去確認最後一筆的日期，再接續向後新增
                if info_list[0] > stock_df.values[-1][0] :
                    df = pd.DataFrame([info_list],columns = columns) # 每一行建立df
                    stock_df = stock_df.append(df,ignore_index = True)  # 把每一行的df 添加到mt_df。 ignore_index = True 不重新編排序號
            else:  # 新建立的df直接添加
                df = pd.DataFrame([info_list],columns = columns) # 每一行建立df
                stock_df = stock_df.append(df,ignore_index = True)  # 把每一行的df 添加到mt_df。 ignore_index = True 不重新編排序號
            
        truncate(io_csv_info) # 清除舊有的記憶體資料
        time.sleep(7)
    #走完一隻股票要去做存檔的動作:
    stock_df.to_csv(save_file_path + f'/{stock}.csv')  #, encoding = 'utf-8-sig')
    print(f'最後日期 {stock_df.values[-1][0]} 已存檔')
    print(f'=====================finished {stock} crawl========================================')
    stock_df.drop(stock_df.index, inplace=True)  # 清除 stock_df，下一隻才不會用到上一檔的資料
    